In [1]:
# Import dependencies
import pandas as pd
%matplotlib notebook 
import matplotlib.pyplot as plt

In [2]:
# Import merged_resedential_electricity_income.csv
file = 'HEAT_mean_2010_merged_income.csv'
income_therm_mean = pd.read_csv(file)

FileNotFoundError: File b'HEAT_mean_2010_merged_income.csv' does not exist

In [ ]:
# Group income_kwh_mean by 'COMMUNITY AREA NAME'
area_income_mean = income_therm_mean.groupby('COMMUNITY AREA NAME').mean()

In [ ]:
# Reset index of area_income_mean
# Clean up DataFrame and drop 'Unnamed: 0' column
area_income_mean.reset_index(inplace=True)
area_income_mean.drop(columns='Unnamed: 0', inplace=True)
area_income_mean.sort_values(by='PER CAPITA INCOME', inplace=True)

# Find the min and max of the mean per capita incomes of each neighborhood
area_income_mean_sum = area_income_mean.describe()
area_income_mean_sum

In [ ]:
# Create bins to group results into per capita income brackets
bins = [0, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000]

# Create names for the bins
incomes = ["0 to 10k", "10k to 20k", "20k to 30k", "30k to 40k",
           "40k to 50k", "50k to 60k", "60k to 70k", "70k to 80k",
           "80k to 90k"]

In [ ]:
# Slice area_income_mean and place it into income bins
# Place data series into a new column inside of area_income_mean
area_income_mean['PER CAPITA INCOME GROUPS'] = pd.cut(area_income_mean["PER CAPITA INCOME"], bins, labels=incomes)

In [ ]:
# Make note of the overview of 'THERM MEAN 2010' for all neighborhoods
area_income_mean['THERM MEAN 2010'].describe()

In [ ]:
# Group area_income_mean to reflect income groups
per_capita_income_grp = area_income_mean.groupby("PER CAPITA INCOME GROUPS").mean()

# Check per_capita_income_grp DataFrame
per_capita_income_grp = per_capita_income_grp[['THERM MEAN 2010']]
per_capita_income_grp

In [ ]:
# Reset index of per_capita_incom_grp
# (will combine with other DataFrams later)
# Make note of how 'THERM MEAN 2010' changed for income brackets
per_capita_income_grp.reset_index(inplace=True)
per_capita_income_grp['THERM MEAN 2010'].describe()

In [ ]:
# Check neighborhood count per income bracket
count_per_capita_income = area_income_mean.groupby("PER CAPITA INCOME GROUPS").count()

# Just take one column from count_per_capita_income to get count
count_per_capita_income = count_per_capita_income[['PER CAPITA INCOME']]

# Reset index to merge DataFrame later
# Rename column to reflect actual data
count_per_capita_income.reset_index(inplace=True)
count_per_capita_income.rename(columns={'PER CAPITA INCOME': 'NEIGHBORHOOD COUNT'}, inplace=True)
count_per_capita_income.head(2)

In [ ]:
# Check total population
# Taking the sum of total population within income brackets
## from the mean total population per community area
total_population = area_income_mean.groupby("PER CAPITA INCOME GROUPS").sum()

# Only select 'TOTAL POPULATION' column
# Reset indext, so DataFrame can be merged later
# Check total_population DataFrame
total_population = total_population[['TOTAL POPULATION']]
total_population.reset_index(inplace=True)
total_population.head(2)

In [ ]:
# Combine DataFrames:
# per_capita_income_grp, count_per_capita_income, total_population

summary_meanKWH_residential = per_capita_income_grp
summary_meanKWH_residential['COMMUNITY AREA COUNT'] = count_per_capita_income['NEIGHBORHOOD COUNT']
summary_meanKWH_residential['MEAN TOTAL POPULATION'] = total_population['TOTAL POPULATION']

summary_meanKWH_residential

In [ ]:
# Create bar plot from summary_meanKWH_residential 
# Compare income brackets to THERM MEAN 2010

# Create plot for per_capita_income_grp 
x = 'PER CAPITA INCOME GROUPS'
y = 'THERM MEAN 2010'
title = 'Chicago - Avg Natural Gas Usage 2010, Grouped by Per Capita Income Brackets'
per_capita_income_grp.plot(kind='bar', x=x, y=y, figsize=(7,10),
                           rot=40, title=title)

# Add y-axix lines to make reading graph easier
plt.gca().yaxis.grid(True, linestyle='--')

# Adjust location of legend
plt.legend(loc='upper left')

# y Label
# 1 Therm = 100,000 British Thermal Units (BTUs)
# Source: https://accel.peoplesgasdelivery.com/home/gas_rates.aspx 
plt.ylabel('Natural Gas Used (therms)')
plt.tight_layout

In [ ]:
# Check income brackets of interest for community area names and PCI
for index, row in area_income_mean.iterrows():
    income = row['PER CAPITA INCOME']
    area = row['COMMUNITY AREA NAME']
    if income < 70000 and income > 60000:
        print(f"Community areas with PCI 60k to 70k")
        print(f"{area} per capita income is {income}")
        print(f"-------------------------------------")
    if income > 80000:
        print(f"Community areas with PCI >80k")
        print(f"{area} per capita income is {income}")
        print(f"-------------------------------------")
    if income < 10000:
        print(f"Community areas with PCI <10k")
        print(f"{area} per capita income is {income}")
        print(f"-------------------------------------")

In [ ]:
# Save figure
plt.savefig('../Graphs/HEAT_Avg_Per_Capita_Income.png')